A CNN on the global views 500x500

In [1]:
import tensorflow as tf
import time
import matplotlib.pyplot as plt
%matplotlib inline

from helpers.data_loading import *
from helpers.CNN_layers import ConvLayer, DenseLayer
from spearman import score_function

In [2]:
y, _ = load_labels()  # scores between 0 and 25 for training instances
y = y.reshape((-1,1))

In [75]:
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement=True
sess = tf.InteractiveSession(config=config)

input_tf = tf.placeholder(tf.float32, shape=[None, 500, 500, 1])
y_tf = tf.placeholder(tf.float32, shape=[None, 1])

conv1 = ConvLayer(input_tf, 8)
conv1.max_pool()

conv2 = ConvLayer(conv1.output, 16)
conv2.max_pool()

conv3 = ConvLayer(conv2.output, 32)
conv3.max_pool()

fc1 = DenseLayer(conv3.output, 256)
fc1.drop_out()

fc2 = DenseLayer(fc1.output, 1)

final_output = 24. * (1 / (1 + tf.exp(- fc2.output)))  # logit to normalize

In [77]:
mse = tf.reduce_mean((final_output - y_tf)**2)
learning_rate = tf.placeholder(tf.float32, shape=[])
train_step = tf.train.AdamOptimizer(learning_rate).minimize(mse)

In [78]:
sess.run(tf.global_variables_initializer())

In [79]:
mse.eval(feed_dict={
    input_tf: global_views_batch(np.arange(1, 51))[0].reshape((-1, 500, 500, 1)), 
    y_tf: y[0:50,:], 
    fc1.keep_prob: 1.0
})

120.84

In [41]:
type(global_views_batch(np.arange(1, 11))[0][0,0,0])

numpy.float32

In [42]:
# WARNING indices for y and when calling global_views_batch differ from 1
# we create a quick batch generator for the training and the testing

# sample train/test indices # WARNING indexed from 1!!!
train = np.random.choice(range(1, 10**4), size = 9500, replace = False)
test = np.setdiff1d(np.arange(1, 10**4), train)

def train_batch(batch_size=50):
    nb_batch = len(train) // batch_size
    while 1:
        for i in range(nb_batch):
            in_batch = train[i*batch_size:((i+1)*batch_size)]
            X_batch, _ = global_views_batch(in_batch)
            y_batch = y[in_batch-1]  # WARNING Y is indexed from 0!!!
            y_batch = y_batch.astype(np.float32)
            yield X_batch, y_batch
            
def test_batch(batch_size=50):
    nb_batch = len(test) // batch_size
    while 1:
        for i in range(nb_batch):
            in_batch = test[i*batch_size:((i+1)*batch_size)]
            X_batch, _ = global_views_batch(in_batch)
            y_batch = y[in_batch-1]  # WARNING Y is indexed from 0!!!
            y_batch = y_batch.astype(np.float32)
            yield X_batch, y_batch

In [69]:
train_gen = train_batch(10)
test_gen = test_batch(10)

In [70]:
X_test, y_test = next(test_gen)
X_test = X_test.reshape((-1, 500, 500, 1))
print X_test.shape
print y_test.shape

(10, 500, 500, 1)
(10, 1)


In [71]:
def epoch(n_ep, lr=1e-4):
    X_train, y_train = next(train_gen)
    X_train = X_train.reshape((-1, 500, 500, 1))
    X_test, y_test = next(test_gen)
    X_test = X_test.reshape((-1, 500, 500, 1))
    train_step.run(feed_dict={
        input_tf: X_train, 
        y_tf: y_train,
        fc1.keep_prob: 0.5,
        learning_rate: lr
    })
    train_mse = sess.run(mse, feed_dict={input_tf:X_train, y_tf: y_train, fc1.keep_prob: 1.0})
    print "iteration %d, training mse %g"%(n_ep, train_mse)
    
    # preds on test
    preds = final_output.eval(feed_dict={input_tf: X_test, fc1.keep_prob: 1.0})
    print preds.flatten()
    print "test mse %g"%np.mean((preds - y_test)**2)
    current_spearman = score_function(y_pred=np.round(preds), y_true=y_test)
    print "test spearman %g"%current_spearman

In [83]:
for i in range(20):
    epoch(1, lr=10**-2)

iteration 1, training mse 116.1
[ 24.  24.  12.  24.  24.  24.  24.  24.  12.  24.]
test mse 99
test spearman 0.371391
iteration 1, training mse 59.2
[ 24.  24.  24.  24.  24.  24.  24.  12.  24.  12.]
test mse 95.7
test spearman 0
iteration 1, training mse 83.6983
[ 24.  12.  12.  24.  24.  12.  24.  24.  12.  12.]
test mse 71.1
test spearman 0.490399
iteration 1, training mse 173.2
[ 24.  24.  24.  24.  12.  12.  24.  24.  24.  24.]
test mse 161
test spearman -0.576328
iteration 1, training mse 98.5
[ 12.  24.  24.  24.  24.  24.  24.  24.  24.  12.]
test mse 83.1
test spearman -0.0909843
iteration 1, training mse 111.591
[ 24.  24.  12.  12.  12.  12.  24.  12.  24.  24.]
test mse 75.6
test spearman -0.175682
iteration 1, training mse 13.2
[ 12.  12.  12.  12.  12.  12.  12.  12.  12.  12.]
test mse 26.1
test spearman nan
iteration 1, training mse 25.7
[ 12.  12.  12.  12.  12.  12.  12.  12.  12.  12.]
test mse 31.4
test spearman nan
iteration 1, training mse 32.3
[ 12.  12.  12.  

In [80]:
final_output.eval(feed_dict={input_tf: X_test.reshape((-1, 500, 500, 1)), fc1.keep_prob: 1.}).flatten()

array([ 12.,  12.,  12.,  24.,  24.,  12.,  24.,  12.,  24.,  24.], dtype=float32)